In [1]:
# -*- coding: utf-8 -*-
# 文件名: solve_q2_by_risk_coefficient.py
# 功能: 通过调整风险系数a，生成一系列候选最优解。
# 版本: v1.0

import pandas as pd
import numpy as np
import os
import re
import random
import copy
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# --- 1. 【核心配置区】 ---
#  在每次运行时，请修改下面的风险系数 a
#  建议取值: 0.1, 0.3, 0.5, 0.7, 0.9
RISK_A_COEFFICIENT = 0.1  # <--- 在这里修改风险系数a (0.0 到 1.0之间)
# --- ---------------- ---


# (1) 遗传算法参数
POP_SIZE_PER_SUBPOP = 50
NUM_POPULATIONS = 5
MAX_GEN = 200
CX_PROB = 0.8
MUT_PROB = 0.2
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5

# (2) 多种群特定参数
MIGRATION_INTERVAL = 25
MIGRATION_SIZE = 3

# (3) 问题二特定参数
NUM_SCENARIOS = 100
YEARS = list(range(2024, 2031))

# --- 2. 数据加载与情景生成 (函数已折叠，内容与之前版本相同) ---
def load_and_prepare_data(data_path):
    try:
        print("（1）正在读取Excel文件...")
        path_f1 = data_path / '附件1.xlsx'
        path_f2 = data_path / '附件2.xlsx'

        plots_df = pd.read_excel(path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(path_f1, sheet_name='乡村种植的农作物')
        stats_df = pd.read_excel(path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(path_f2, sheet_name='2023年的农作物种植情况')

        for df in [plots_df, crops_info_df, stats_df, past_planting_df]:
            df.columns = df.columns.str.strip()

        params = {}
        params['I_plots'] = sorted(plots_df['地块名称'].tolist())
        params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
        params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))

        params['J_crops'] = sorted(crops_info_df['作物名称'].dropna().unique().tolist())
        params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
        params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and '豆' in ctype]

        params['P_past'] = {i: {1: None, 2: None} for i in params['I_plots']}
        for _, row in past_planting_df.iterrows():
            plot, crop = row['种植地块'], row['作物名称']
            season = row.get('种植季节', 1)
            if plot in params['I_plots']:
                params['P_past'][plot][season] = crop

        def clean_and_convert_price(value):
            if isinstance(value, str) and any(c in value for c in '-–—'):
                parts = re.split(r'[-–—]', value.strip())
                try: return (float(parts[0]) + float(parts[1])) / 2
                except (ValueError, IndexError): return np.nan
            return pd.to_numeric(value, errors='coerce')

        stats_df['销售单价/(元/斤)'] = stats_df['销售单价/(元/斤)'].apply(clean_and_convert_price)
        stats_df.dropna(subset=['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)'], inplace=True)
        
        params['P_yield_base'], params['P_cost_base'], params['P_price_base'] = {}, {}, {}
        for _, row in stats_df.iterrows():
            key = (row['作物名称'], row['地块类型'])
            params['P_cost_base'][key] = row['种植成本/(元/亩)']
            params['P_yield_base'][key] = row['亩产量/斤']
            params['P_price_base'][key] = row['销售单价/(元/斤)']

        params['P_demand_base'] = {j: 0 for j in params['J_crops']}
        merged_df = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称', how='left')
        
        for crop in params['J_crops']:
            crop_plantings = merged_df[merged_df['作物名称'] == crop]
            total_yield = 0
            if not crop_plantings.empty:
                for _, row in crop_plantings.iterrows():
                    area = row.get('种植面积/亩', params['P_area'][row['种植地块']])
                    yield_val = params['P_yield_base'].get((crop, row['地块类型']), 0)
                    total_yield += area * yield_val
            params['P_demand_base'][crop] = total_yield

        params['S_suitability'] = {}
        restricted_veg = ['大白菜', '白萝卜', '红萝卜']
        for i in params['I_plots']:
            plot_t = params['P_plot_type'].get(i, '')
            for j in params['J_crops']:
                crop_t = params['P_crop_type'].get(j, '')
                is_veg = '蔬菜' in str(crop_t)
                for k in [1, 2]:
                    suitable = 0
                    if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in str(crop_t) or j in params['J_bean']) and k == 1: suitable = 1
                    elif plot_t == '水浇地':
                        if '水稻' in j and k == 1: suitable = 1
                        elif is_veg:
                            if j not in restricted_veg and k == 1: suitable = 1
                            elif j in restricted_veg and k == 2: suitable = 1
                    elif plot_t == '普通大棚':
                        if is_veg and j not in restricted_veg and k == 1: suitable = 1
                        elif '食用菌' in str(crop_t) and k == 2: suitable = 1
                    elif plot_t == '智慧大棚' and is_veg and j not in restricted_veg: suitable = 1
                    params['S_suitability'][(i, j, k)] = suitable
        
        print(" -> 基础数据参数准备完成。")
        return params
    except Exception as e:
        print(f"错误: 加载数据失败: {e}"); raise

def generate_scenarios(params, num_scenarios):
    print(f"（2）正在生成 {num_scenarios} 个未来情景...")
    scenarios = []
    crop_types = params['P_crop_type']
    base_demand = params['P_demand_base']
    base_yield = params['P_yield_base']
    base_cost = params['P_cost_base']
    base_price = params['P_price_base']
    for _ in tqdm(range(num_scenarios), desc="生成情景"):
        scenario = {
            'P_demand': {y: {} for y in YEARS}, 'P_yield': {y: {} for y in YEARS},
            'P_cost': {y: {} for y in YEARS}, 'P_price': {y: {} for y in YEARS},
        }
        temp_demand = copy.deepcopy(base_demand)
        temp_cost = copy.deepcopy(base_cost)
        temp_price = copy.deepcopy(base_price)
        for y in YEARS:
            for crop in params['J_crops']:
                if y > 2024:
                    if crop in ['小麦', '玉米']: growth_rate = np.random.uniform(0.05, 0.10)
                    else: growth_rate = np.random.uniform(-0.05, 0.05)
                    temp_demand[crop] *= (1 + growth_rate)
            scenario['P_demand'][y] = copy.deepcopy(temp_demand)
            for key, b_yield in base_yield.items():
                scenario['P_yield'][y][key] = b_yield * (1 + np.random.uniform(-0.10, 0.10))
            if y > 2024:
                for key in temp_cost: temp_cost[key] *= (1 + np.random.normal(0.05, 0.01))
            scenario['P_cost'][y] = copy.deepcopy(temp_cost)
            if y > 2024:
                for key in temp_price:
                    crop, _ = key
                    ctype = crop_types.get(crop, '')
                    if '蔬菜' in str(ctype): temp_price[key] *= (1 + np.random.normal(0.05, 0.02))
                    elif '食用菌' in str(ctype): temp_price[key] *= (1 - (0.05 if crop == '羊肚菌' else np.random.uniform(0.01, 0.05)))
            scenario['P_price'][y] = copy.deepcopy(temp_price)
        scenarios.append(scenario)
    print(" -> 情景生成完毕。")
    return scenarios

# --- 3. 遗传算法核心函数 ---
def create_initial_solution(params):
    solution = {y: {k: {i: None for i in params['I_plots']} for k in [1, 2]} for y in YEARS}
    for y in YEARS:
        for i in params['I_plots']:
            for k in [1, 2]:
                possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
                if possible_crops:
                    solution[y][k][i] = random.choice(possible_crops)
    return repair_solution(solution, params)

def repair_solution(solution, params):
    def get_crops_in_year(sol, y, i):
        crops = set()
        if y == 2023:
            for k in [1, 2]:
                crop = params['P_past'].get(i, {}).get(k)
                if crop: crops.add(crop)
        elif y in sol:
            for k in [1, 2]:
                crop = sol.get(y, {}).get(k, {}).get(i)
                if crop: crops.add(crop)
        return list(crops)
    for i in params['I_plots']:
        for y in YEARS:
            crops_last_year = get_crops_in_year(solution, y - 1, i)
            for k in [1, 2]:
                crop_this_season = solution[y][k][i]
                if crop_this_season and crop_this_season in crops_last_year:
                    possible_replacements = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1 and j not in crops_last_year]
                    solution[y][k][i] = random.choice(possible_replacements) if possible_replacements else None
    for i in params['I_plots']:
        all_years = [2023] + YEARS
        for idx in range(len(all_years) - 2):
            window = all_years[idx:idx+3]
            contains_bean = any(c in params['J_bean'] for y_win in window for c in get_crops_in_year(solution, y_win, i))
            if not contains_bean:
                y_fix = random.choice([y for y in window if y > 2023])
                k_fix = 1
                crops_last_year = get_crops_in_year(solution, y_fix - 1, i)
                possible_beans = [b for b in params['J_bean'] if params['S_suitability'].get((i, b, k_fix), 0) == 1 and b not in crops_last_year]
                if possible_beans:
                    solution[y_fix][k_fix][i] = random.choice(possible_beans)
    return solution

def crossover(p1, p2, params):
    child = copy.deepcopy(p1)
    for i in params['I_plots']:
        if random.random() < 0.5:
            for y in YEARS:
                for k in [1, 2]:
                    child[y][k][i] = p2[y][k][i]
    return child

def mutate(solution, params):
    mut_sol = copy.deepcopy(solution)
    for _ in range(random.randint(1, 5)):
        y = random.choice(YEARS)
        i = random.choice(params['I_plots'])
        k = random.choice([1, 2])
        possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
        if possible_crops:
            mut_sol[y][k][i] = random.choice(possible_crops)
    return mut_sol

def calculate_profits_for_solution(solution, params, scenarios):
    profits = []
    for scenario in scenarios:
        total_revenue, total_cost = 0, 0
        total_production_by_crop = defaultdict(float)
        for y in YEARS:
            for i in params['I_plots']:
                plot_type = params['P_plot_type'][i]
                area = params['P_area'][i]
                for k in [1, 2]:
                    crop = solution[y][k].get(i)
                    if not crop: continue
                    key = (crop, plot_type)
                    cost = scenario['P_cost'][y].get(key, 9e9)
                    yield_val = scenario['P_yield'][y].get(key, 0)
                    if cost > 1e9: continue
                    total_cost += area * cost
                    total_production_by_crop[crop] += area * yield_val
        for crop, production in total_production_by_crop.items():
            total_demand_7_years = sum(scenario['P_demand'][y].get(crop, 0) for y in YEARS)
            all_prices = [p for y_prices in scenario['P_price'].values() for (c, pt), p in y_prices.items() if c == crop and p > 0]
            price = np.mean(all_prices) if all_prices else 0
            if price > 0:
                normal_qty = min(production, total_demand_7_years)
                over_qty = production - normal_qty
                total_revenue += (normal_qty * price) + (over_qty * price * 0.5)
        profits.append(total_revenue - total_cost)
    return profits

def evaluate_fitness_q2(solution, params, scenarios, a_coeff):
    """
    【关键修改】适应度函数：使用 a*E(X) - (1-a)*σ(X)
    """
    profits = calculate_profits_for_solution(solution, params, scenarios)
    valid_profits = [p for p in profits if not np.isnan(p)]
    if not valid_profits: return -1e9

    expected_profit = np.mean(valid_profits)
    risk = np.std(valid_profits)
    
    # a_coeff 是收益的权重, (1-a_coeff) 是风险的权重
    fitness = a_coeff * expected_profit - (1 - a_coeff) * risk
    return fitness

# --- 4. 多种群遗传算法(MPGA)运行器 ---
def run_mpga(params, scenarios, a_coeff):
    print(f"\n--- 开始执行MPGA (风险系数 a = {a_coeff}) ---")
    populations = [[create_initial_solution(params) for _ in range(POP_SIZE_PER_SUBPOP)] for _ in range(NUM_POPULATIONS)]
    best_solution_overall, best_fitness_overall = None, -np.inf
    convergence_history = []
    for gen in tqdm(range(MAX_GEN), desc=f"MPGA进化中 (a={a_coeff})"):
        all_fitnesses = []
        for i in range(NUM_POPULATIONS):
            pop = populations[i]
            fitnesses = [evaluate_fitness_q2(sol, params, scenarios, a_coeff) for sol in pop]
            all_fitnesses.append(fitnesses)
            best_fit_in_pop = np.max(fitnesses)
            if best_fit_in_pop > best_fitness_overall:
                best_fitness_overall = best_fit_in_pop
                best_solution_overall = copy.deepcopy(pop[np.argmax(fitnesses)])
            elite_indices = np.argsort(fitnesses)[-ELITISM_SIZE:]
            new_pop = [pop[idx] for idx in elite_indices]
            while len(new_pop) < POP_SIZE_PER_SUBPOP:
                def tournament_selection(p, f, k):
                    best = random.randrange(len(p))
                    for _ in range(k - 1):
                        idx = random.randrange(len(p))
                        if f[idx] > f[best]: best = idx
                    return p[best]
                p1 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                p2 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                child = crossover(p1, p2, params) if random.random() < CX_PROB else copy.deepcopy(p1)
                if random.random() < MUT_PROB:
                    child = mutate(child, params)
                new_pop.append(repair_solution(child, params))
            populations[i] = new_pop
        if gen > 0 and gen % MIGRATION_INTERVAL == 0:
            for i in range(NUM_POPULATIONS):
                target_pop_idx = (i + 1) % NUM_POPULATIONS
                best_indices_current = np.argsort(all_fitnesses[i])[-MIGRATION_SIZE:]
                migrants = [populations[i][idx] for idx in best_indices_current]
                worst_indices_target = np.argsort(all_fitnesses[target_pop_idx])[:MIGRATION_SIZE]
                for j in range(MIGRATION_SIZE):
                    populations[target_pop_idx][worst_indices_target[j]] = copy.deepcopy(migrants[j])
        convergence_history.append({'Generation': gen, 'Best_Fitness': best_fitness_overall})
    print(f"\n--- MPGA (a={a_coeff}) 优化完成 ---")
    return best_solution_overall, best_fitness_overall, convergence_history

# --- 5. 主程序 ---
if __name__ == '__main__':
    try:
        a = RISK_A_COEFFICIENT
        if not (0.0 <= a <= 1.0):
            raise ValueError("错误：风险系数 a 必须在 0.0 和 1.0 之间。")

        script_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
        data_path = script_dir / '..' / 'Data'
        output_dir = script_dir / 'Result'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"脚本运行目录: {script_dir}")
        print(f"数据读取路径: {data_path}")
        print(f"结果输出路径: {output_dir}")
        
        base_params = load_and_prepare_data(data_path)
        scenarios = generate_scenarios(base_params, NUM_SCENARIOS)

        best_solution, best_fitness, history = run_mpga(base_params, scenarios, a)
        
        print(f"\n求解完成 (风险系数 a = {a})。")
        print(f" -> 最优适应度值: {best_fitness:,.2f}")

        if best_solution:
            file_suffix = f"a{str(a).replace('.', '')}" # e.g., a=0.7 -> a07

            # (1) 保存最终方案
            output_list = []
            for y in sorted(best_solution.keys()):
                for k in sorted(best_solution[y].keys()):
                    for i in sorted(best_solution[y][k].keys()):
                        crop = best_solution[y][k][i]
                        if crop:
                            output_list.append({'年份': y, '季节': k, '地块编号': i, '作物名称': crop, '种植面积（亩）': base_params['P_area'][i]})
            result_df = pd.DataFrame(output_list)
            file_path = output_dir / f'result2_{file_suffix}.xlsx'
            result_df.to_excel(file_path, index=False)
            print(f"最优方案已保存至: {file_path}")

            # (2) 保存收敛过程数据
            history_df = pd.DataFrame(history)
            log_path = output_dir / f'log_{file_suffix}.csv'
            history_df.to_csv(log_path, index=False)
            print(f"收敛过程日志已保存至: {log_path}")

            # (3) 保存最优方案的利润分布
            final_profits = calculate_profits_for_solution(best_solution, base_params, scenarios)
            profits_df = pd.DataFrame(final_profits, columns=['Profit'])
            profits_path = output_dir / f'profits_{file_suffix}.csv'
            profits_df.to_csv(profits_path, index=False)
            print(f"最优方案利润分布已保存至: {profits_path}")
        else:
            print("未能找到有效解。")

    except Exception as e:
        print(f"\n程序主流程发生错误: {e}")
        import traceback
        traceback.print_exc()

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
# -*- coding: utf-8 -*-
# 文件名: solve_q2_by_risk_coefficient.py
# 功能: 通过调整风险系数a，生成一系列候选最优解。
# 版本: v1.0

import pandas as pd
import numpy as np
import os
import re
import random
import copy
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# --- 1. 【核心配置区】 ---
#  在每次运行时，请修改下面的风险系数 a
#  建议取值: 0.1, 0.3, 0.5, 0.7, 0.9
RISK_A_COEFFICIENT = 0.3  # <--- 在这里修改风险系数a (0.0 到 1.0之间)
# --- ---------------- ---


# (1) 遗传算法参数
POP_SIZE_PER_SUBPOP = 50
NUM_POPULATIONS = 5
MAX_GEN = 200
CX_PROB = 0.8
MUT_PROB = 0.2
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5

# (2) 多种群特定参数
MIGRATION_INTERVAL = 25
MIGRATION_SIZE = 3

# (3) 问题二特定参数
NUM_SCENARIOS = 100
YEARS = list(range(2024, 2031))

# --- 2. 数据加载与情景生成 (函数已折叠，内容与之前版本相同) ---
def load_and_prepare_data(data_path):
    try:
        print("（1）正在读取Excel文件...")
        path_f1 = data_path / '附件1.xlsx'
        path_f2 = data_path / '附件2.xlsx'

        plots_df = pd.read_excel(path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(path_f1, sheet_name='乡村种植的农作物')
        stats_df = pd.read_excel(path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(path_f2, sheet_name='2023年的农作物种植情况')

        for df in [plots_df, crops_info_df, stats_df, past_planting_df]:
            df.columns = df.columns.str.strip()

        params = {}
        params['I_plots'] = sorted(plots_df['地块名称'].tolist())
        params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
        params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))

        params['J_crops'] = sorted(crops_info_df['作物名称'].dropna().unique().tolist())
        params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
        params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and '豆' in ctype]

        params['P_past'] = {i: {1: None, 2: None} for i in params['I_plots']}
        for _, row in past_planting_df.iterrows():
            plot, crop = row['种植地块'], row['作物名称']
            season = row.get('种植季节', 1)
            if plot in params['I_plots']:
                params['P_past'][plot][season] = crop

        def clean_and_convert_price(value):
            if isinstance(value, str) and any(c in value for c in '-–—'):
                parts = re.split(r'[-–—]', value.strip())
                try: return (float(parts[0]) + float(parts[1])) / 2
                except (ValueError, IndexError): return np.nan
            return pd.to_numeric(value, errors='coerce')

        stats_df['销售单价/(元/斤)'] = stats_df['销售单价/(元/斤)'].apply(clean_and_convert_price)
        stats_df.dropna(subset=['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)'], inplace=True)
        
        params['P_yield_base'], params['P_cost_base'], params['P_price_base'] = {}, {}, {}
        for _, row in stats_df.iterrows():
            key = (row['作物名称'], row['地块类型'])
            params['P_cost_base'][key] = row['种植成本/(元/亩)']
            params['P_yield_base'][key] = row['亩产量/斤']
            params['P_price_base'][key] = row['销售单价/(元/斤)']

        params['P_demand_base'] = {j: 0 for j in params['J_crops']}
        merged_df = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称', how='left')
        
        for crop in params['J_crops']:
            crop_plantings = merged_df[merged_df['作物名称'] == crop]
            total_yield = 0
            if not crop_plantings.empty:
                for _, row in crop_plantings.iterrows():
                    area = row.get('种植面积/亩', params['P_area'][row['种植地块']])
                    yield_val = params['P_yield_base'].get((crop, row['地块类型']), 0)
                    total_yield += area * yield_val
            params['P_demand_base'][crop] = total_yield

        params['S_suitability'] = {}
        restricted_veg = ['大白菜', '白萝卜', '红萝卜']
        for i in params['I_plots']:
            plot_t = params['P_plot_type'].get(i, '')
            for j in params['J_crops']:
                crop_t = params['P_crop_type'].get(j, '')
                is_veg = '蔬菜' in str(crop_t)
                for k in [1, 2]:
                    suitable = 0
                    if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in str(crop_t) or j in params['J_bean']) and k == 1: suitable = 1
                    elif plot_t == '水浇地':
                        if '水稻' in j and k == 1: suitable = 1
                        elif is_veg:
                            if j not in restricted_veg and k == 1: suitable = 1
                            elif j in restricted_veg and k == 2: suitable = 1
                    elif plot_t == '普通大棚':
                        if is_veg and j not in restricted_veg and k == 1: suitable = 1
                        elif '食用菌' in str(crop_t) and k == 2: suitable = 1
                    elif plot_t == '智慧大棚' and is_veg and j not in restricted_veg: suitable = 1
                    params['S_suitability'][(i, j, k)] = suitable
        
        print(" -> 基础数据参数准备完成。")
        return params
    except Exception as e:
        print(f"错误: 加载数据失败: {e}"); raise

def generate_scenarios(params, num_scenarios):
    print(f"（2）正在生成 {num_scenarios} 个未来情景...")
    scenarios = []
    crop_types = params['P_crop_type']
    base_demand = params['P_demand_base']
    base_yield = params['P_yield_base']
    base_cost = params['P_cost_base']
    base_price = params['P_price_base']
    for _ in tqdm(range(num_scenarios), desc="生成情景"):
        scenario = {
            'P_demand': {y: {} for y in YEARS}, 'P_yield': {y: {} for y in YEARS},
            'P_cost': {y: {} for y in YEARS}, 'P_price': {y: {} for y in YEARS},
        }
        temp_demand = copy.deepcopy(base_demand)
        temp_cost = copy.deepcopy(base_cost)
        temp_price = copy.deepcopy(base_price)
        for y in YEARS:
            for crop in params['J_crops']:
                if y > 2024:
                    if crop in ['小麦', '玉米']: growth_rate = np.random.uniform(0.05, 0.10)
                    else: growth_rate = np.random.uniform(-0.05, 0.05)
                    temp_demand[crop] *= (1 + growth_rate)
            scenario['P_demand'][y] = copy.deepcopy(temp_demand)
            for key, b_yield in base_yield.items():
                scenario['P_yield'][y][key] = b_yield * (1 + np.random.uniform(-0.10, 0.10))
            if y > 2024:
                for key in temp_cost: temp_cost[key] *= (1 + np.random.normal(0.05, 0.01))
            scenario['P_cost'][y] = copy.deepcopy(temp_cost)
            if y > 2024:
                for key in temp_price:
                    crop, _ = key
                    ctype = crop_types.get(crop, '')
                    if '蔬菜' in str(ctype): temp_price[key] *= (1 + np.random.normal(0.05, 0.02))
                    elif '食用菌' in str(ctype): temp_price[key] *= (1 - (0.05 if crop == '羊肚菌' else np.random.uniform(0.01, 0.05)))
            scenario['P_price'][y] = copy.deepcopy(temp_price)
        scenarios.append(scenario)
    print(" -> 情景生成完毕。")
    return scenarios

# --- 3. 遗传算法核心函数 ---
def create_initial_solution(params):
    solution = {y: {k: {i: None for i in params['I_plots']} for k in [1, 2]} for y in YEARS}
    for y in YEARS:
        for i in params['I_plots']:
            for k in [1, 2]:
                possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
                if possible_crops:
                    solution[y][k][i] = random.choice(possible_crops)
    return repair_solution(solution, params)

def repair_solution(solution, params):
    def get_crops_in_year(sol, y, i):
        crops = set()
        if y == 2023:
            for k in [1, 2]:
                crop = params['P_past'].get(i, {}).get(k)
                if crop: crops.add(crop)
        elif y in sol:
            for k in [1, 2]:
                crop = sol.get(y, {}).get(k, {}).get(i)
                if crop: crops.add(crop)
        return list(crops)
    for i in params['I_plots']:
        for y in YEARS:
            crops_last_year = get_crops_in_year(solution, y - 1, i)
            for k in [1, 2]:
                crop_this_season = solution[y][k][i]
                if crop_this_season and crop_this_season in crops_last_year:
                    possible_replacements = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1 and j not in crops_last_year]
                    solution[y][k][i] = random.choice(possible_replacements) if possible_replacements else None
    for i in params['I_plots']:
        all_years = [2023] + YEARS
        for idx in range(len(all_years) - 2):
            window = all_years[idx:idx+3]
            contains_bean = any(c in params['J_bean'] for y_win in window for c in get_crops_in_year(solution, y_win, i))
            if not contains_bean:
                y_fix = random.choice([y for y in window if y > 2023])
                k_fix = 1
                crops_last_year = get_crops_in_year(solution, y_fix - 1, i)
                possible_beans = [b for b in params['J_bean'] if params['S_suitability'].get((i, b, k_fix), 0) == 1 and b not in crops_last_year]
                if possible_beans:
                    solution[y_fix][k_fix][i] = random.choice(possible_beans)
    return solution

def crossover(p1, p2, params):
    child = copy.deepcopy(p1)
    for i in params['I_plots']:
        if random.random() < 0.5:
            for y in YEARS:
                for k in [1, 2]:
                    child[y][k][i] = p2[y][k][i]
    return child

def mutate(solution, params):
    mut_sol = copy.deepcopy(solution)
    for _ in range(random.randint(1, 5)):
        y = random.choice(YEARS)
        i = random.choice(params['I_plots'])
        k = random.choice([1, 2])
        possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
        if possible_crops:
            mut_sol[y][k][i] = random.choice(possible_crops)
    return mut_sol

def calculate_profits_for_solution(solution, params, scenarios):
    profits = []
    for scenario in scenarios:
        total_revenue, total_cost = 0, 0
        total_production_by_crop = defaultdict(float)
        for y in YEARS:
            for i in params['I_plots']:
                plot_type = params['P_plot_type'][i]
                area = params['P_area'][i]
                for k in [1, 2]:
                    crop = solution[y][k].get(i)
                    if not crop: continue
                    key = (crop, plot_type)
                    cost = scenario['P_cost'][y].get(key, 9e9)
                    yield_val = scenario['P_yield'][y].get(key, 0)
                    if cost > 1e9: continue
                    total_cost += area * cost
                    total_production_by_crop[crop] += area * yield_val
        for crop, production in total_production_by_crop.items():
            total_demand_7_years = sum(scenario['P_demand'][y].get(crop, 0) for y in YEARS)
            all_prices = [p for y_prices in scenario['P_price'].values() for (c, pt), p in y_prices.items() if c == crop and p > 0]
            price = np.mean(all_prices) if all_prices else 0
            if price > 0:
                normal_qty = min(production, total_demand_7_years)
                over_qty = production - normal_qty
                total_revenue += (normal_qty * price) + (over_qty * price * 0.5)
        profits.append(total_revenue - total_cost)
    return profits

def evaluate_fitness_q2(solution, params, scenarios, a_coeff):
    """
    【关键修改】适应度函数：使用 a*E(X) - (1-a)*σ(X)
    """
    profits = calculate_profits_for_solution(solution, params, scenarios)
    valid_profits = [p for p in profits if not np.isnan(p)]
    if not valid_profits: return -1e9

    expected_profit = np.mean(valid_profits)
    risk = np.std(valid_profits)
    
    # a_coeff 是收益的权重, (1-a_coeff) 是风险的权重
    fitness = a_coeff * expected_profit - (1 - a_coeff) * risk
    return fitness

# --- 4. 多种群遗传算法(MPGA)运行器 ---
def run_mpga(params, scenarios, a_coeff):
    print(f"\n--- 开始执行MPGA (风险系数 a = {a_coeff}) ---")
    populations = [[create_initial_solution(params) for _ in range(POP_SIZE_PER_SUBPOP)] for _ in range(NUM_POPULATIONS)]
    best_solution_overall, best_fitness_overall = None, -np.inf
    convergence_history = []
    for gen in tqdm(range(MAX_GEN), desc=f"MPGA进化中 (a={a_coeff})"):
        all_fitnesses = []
        for i in range(NUM_POPULATIONS):
            pop = populations[i]
            fitnesses = [evaluate_fitness_q2(sol, params, scenarios, a_coeff) for sol in pop]
            all_fitnesses.append(fitnesses)
            best_fit_in_pop = np.max(fitnesses)
            if best_fit_in_pop > best_fitness_overall:
                best_fitness_overall = best_fit_in_pop
                best_solution_overall = copy.deepcopy(pop[np.argmax(fitnesses)])
            elite_indices = np.argsort(fitnesses)[-ELITISM_SIZE:]
            new_pop = [pop[idx] for idx in elite_indices]
            while len(new_pop) < POP_SIZE_PER_SUBPOP:
                def tournament_selection(p, f, k):
                    best = random.randrange(len(p))
                    for _ in range(k - 1):
                        idx = random.randrange(len(p))
                        if f[idx] > f[best]: best = idx
                    return p[best]
                p1 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                p2 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                child = crossover(p1, p2, params) if random.random() < CX_PROB else copy.deepcopy(p1)
                if random.random() < MUT_PROB:
                    child = mutate(child, params)
                new_pop.append(repair_solution(child, params))
            populations[i] = new_pop
        if gen > 0 and gen % MIGRATION_INTERVAL == 0:
            for i in range(NUM_POPULATIONS):
                target_pop_idx = (i + 1) % NUM_POPULATIONS
                best_indices_current = np.argsort(all_fitnesses[i])[-MIGRATION_SIZE:]
                migrants = [populations[i][idx] for idx in best_indices_current]
                worst_indices_target = np.argsort(all_fitnesses[target_pop_idx])[:MIGRATION_SIZE]
                for j in range(MIGRATION_SIZE):
                    populations[target_pop_idx][worst_indices_target[j]] = copy.deepcopy(migrants[j])
        convergence_history.append({'Generation': gen, 'Best_Fitness': best_fitness_overall})
    print(f"\n--- MPGA (a={a_coeff}) 优化完成 ---")
    return best_solution_overall, best_fitness_overall, convergence_history

# --- 5. 主程序 ---
if __name__ == '__main__':
    try:
        a = RISK_A_COEFFICIENT
        if not (0.0 <= a <= 1.0):
            raise ValueError("错误：风险系数 a 必须在 0.0 和 1.0 之间。")

        script_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
        data_path = script_dir / '..' / 'Data'
        output_dir = script_dir / 'Result'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"脚本运行目录: {script_dir}")
        print(f"数据读取路径: {data_path}")
        print(f"结果输出路径: {output_dir}")
        
        base_params = load_and_prepare_data(data_path)
        scenarios = generate_scenarios(base_params, NUM_SCENARIOS)

        best_solution, best_fitness, history = run_mpga(base_params, scenarios, a)
        
        print(f"\n求解完成 (风险系数 a = {a})。")
        print(f" -> 最优适应度值: {best_fitness:,.2f}")

        if best_solution:
            file_suffix = f"a{str(a).replace('.', '')}" # e.g., a=0.7 -> a07

            # (1) 保存最终方案
            output_list = []
            for y in sorted(best_solution.keys()):
                for k in sorted(best_solution[y].keys()):
                    for i in sorted(best_solution[y][k].keys()):
                        crop = best_solution[y][k][i]
                        if crop:
                            output_list.append({'年份': y, '季节': k, '地块编号': i, '作物名称': crop, '种植面积（亩）': base_params['P_area'][i]})
            result_df = pd.DataFrame(output_list)
            file_path = output_dir / f'result2_{file_suffix}.xlsx'
            result_df.to_excel(file_path, index=False)
            print(f"最优方案已保存至: {file_path}")

            # (2) 保存收敛过程数据
            history_df = pd.DataFrame(history)
            log_path = output_dir / f'log_{file_suffix}.csv'
            history_df.to_csv(log_path, index=False)
            print(f"收敛过程日志已保存至: {log_path}")

            # (3) 保存最优方案的利润分布
            final_profits = calculate_profits_for_solution(best_solution, base_params, scenarios)
            profits_df = pd.DataFrame(final_profits, columns=['Profit'])
            profits_path = output_dir / f'profits_{file_suffix}.csv'
            profits_df.to_csv(profits_path, index=False)
            print(f"最优方案利润分布已保存至: {profits_path}")
        else:
            print("未能找到有效解。")

    except Exception as e:
        print(f"\n程序主流程发生错误: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# -*- coding: utf-8 -*-
# 文件名: solve_q2_by_risk_coefficient.py
# 功能: 通过调整风险系数a，生成一系列候选最优解。
# 版本: v1.0

import pandas as pd
import numpy as np
import os
import re
import random
import copy
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# --- 1. 【核心配置区】 ---
#  在每次运行时，请修改下面的风险系数 a
#  建议取值: 0.1, 0.3, 0.5, 0.7, 0.9
RISK_A_COEFFICIENT = 0.5  # <--- 在这里修改风险系数a (0.0 到 1.0之间)
# --- ---------------- ---


# (1) 遗传算法参数
POP_SIZE_PER_SUBPOP = 50
NUM_POPULATIONS = 5
MAX_GEN = 200
CX_PROB = 0.8
MUT_PROB = 0.2
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5

# (2) 多种群特定参数
MIGRATION_INTERVAL = 25
MIGRATION_SIZE = 3

# (3) 问题二特定参数
NUM_SCENARIOS = 100
YEARS = list(range(2024, 2031))

# --- 2. 数据加载与情景生成 (函数已折叠，内容与之前版本相同) ---
def load_and_prepare_data(data_path):
    try:
        print("（1）正在读取Excel文件...")
        path_f1 = data_path / '附件1.xlsx'
        path_f2 = data_path / '附件2.xlsx'

        plots_df = pd.read_excel(path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(path_f1, sheet_name='乡村种植的农作物')
        stats_df = pd.read_excel(path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(path_f2, sheet_name='2023年的农作物种植情况')

        for df in [plots_df, crops_info_df, stats_df, past_planting_df]:
            df.columns = df.columns.str.strip()

        params = {}
        params['I_plots'] = sorted(plots_df['地块名称'].tolist())
        params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
        params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))

        params['J_crops'] = sorted(crops_info_df['作物名称'].dropna().unique().tolist())
        params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
        params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and '豆' in ctype]

        params['P_past'] = {i: {1: None, 2: None} for i in params['I_plots']}
        for _, row in past_planting_df.iterrows():
            plot, crop = row['种植地块'], row['作物名称']
            season = row.get('种植季节', 1)
            if plot in params['I_plots']:
                params['P_past'][plot][season] = crop

        def clean_and_convert_price(value):
            if isinstance(value, str) and any(c in value for c in '-–—'):
                parts = re.split(r'[-–—]', value.strip())
                try: return (float(parts[0]) + float(parts[1])) / 2
                except (ValueError, IndexError): return np.nan
            return pd.to_numeric(value, errors='coerce')

        stats_df['销售单价/(元/斤)'] = stats_df['销售单价/(元/斤)'].apply(clean_and_convert_price)
        stats_df.dropna(subset=['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)'], inplace=True)
        
        params['P_yield_base'], params['P_cost_base'], params['P_price_base'] = {}, {}, {}
        for _, row in stats_df.iterrows():
            key = (row['作物名称'], row['地块类型'])
            params['P_cost_base'][key] = row['种植成本/(元/亩)']
            params['P_yield_base'][key] = row['亩产量/斤']
            params['P_price_base'][key] = row['销售单价/(元/斤)']

        params['P_demand_base'] = {j: 0 for j in params['J_crops']}
        merged_df = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称', how='left')
        
        for crop in params['J_crops']:
            crop_plantings = merged_df[merged_df['作物名称'] == crop]
            total_yield = 0
            if not crop_plantings.empty:
                for _, row in crop_plantings.iterrows():
                    area = row.get('种植面积/亩', params['P_area'][row['种植地块']])
                    yield_val = params['P_yield_base'].get((crop, row['地块类型']), 0)
                    total_yield += area * yield_val
            params['P_demand_base'][crop] = total_yield

        params['S_suitability'] = {}
        restricted_veg = ['大白菜', '白萝卜', '红萝卜']
        for i in params['I_plots']:
            plot_t = params['P_plot_type'].get(i, '')
            for j in params['J_crops']:
                crop_t = params['P_crop_type'].get(j, '')
                is_veg = '蔬菜' in str(crop_t)
                for k in [1, 2]:
                    suitable = 0
                    if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in str(crop_t) or j in params['J_bean']) and k == 1: suitable = 1
                    elif plot_t == '水浇地':
                        if '水稻' in j and k == 1: suitable = 1
                        elif is_veg:
                            if j not in restricted_veg and k == 1: suitable = 1
                            elif j in restricted_veg and k == 2: suitable = 1
                    elif plot_t == '普通大棚':
                        if is_veg and j not in restricted_veg and k == 1: suitable = 1
                        elif '食用菌' in str(crop_t) and k == 2: suitable = 1
                    elif plot_t == '智慧大棚' and is_veg and j not in restricted_veg: suitable = 1
                    params['S_suitability'][(i, j, k)] = suitable
        
        print(" -> 基础数据参数准备完成。")
        return params
    except Exception as e:
        print(f"错误: 加载数据失败: {e}"); raise

def generate_scenarios(params, num_scenarios):
    print(f"（2）正在生成 {num_scenarios} 个未来情景...")
    scenarios = []
    crop_types = params['P_crop_type']
    base_demand = params['P_demand_base']
    base_yield = params['P_yield_base']
    base_cost = params['P_cost_base']
    base_price = params['P_price_base']
    for _ in tqdm(range(num_scenarios), desc="生成情景"):
        scenario = {
            'P_demand': {y: {} for y in YEARS}, 'P_yield': {y: {} for y in YEARS},
            'P_cost': {y: {} for y in YEARS}, 'P_price': {y: {} for y in YEARS},
        }
        temp_demand = copy.deepcopy(base_demand)
        temp_cost = copy.deepcopy(base_cost)
        temp_price = copy.deepcopy(base_price)
        for y in YEARS:
            for crop in params['J_crops']:
                if y > 2024:
                    if crop in ['小麦', '玉米']: growth_rate = np.random.uniform(0.05, 0.10)
                    else: growth_rate = np.random.uniform(-0.05, 0.05)
                    temp_demand[crop] *= (1 + growth_rate)
            scenario['P_demand'][y] = copy.deepcopy(temp_demand)
            for key, b_yield in base_yield.items():
                scenario['P_yield'][y][key] = b_yield * (1 + np.random.uniform(-0.10, 0.10))
            if y > 2024:
                for key in temp_cost: temp_cost[key] *= (1 + np.random.normal(0.05, 0.01))
            scenario['P_cost'][y] = copy.deepcopy(temp_cost)
            if y > 2024:
                for key in temp_price:
                    crop, _ = key
                    ctype = crop_types.get(crop, '')
                    if '蔬菜' in str(ctype): temp_price[key] *= (1 + np.random.normal(0.05, 0.02))
                    elif '食用菌' in str(ctype): temp_price[key] *= (1 - (0.05 if crop == '羊肚菌' else np.random.uniform(0.01, 0.05)))
            scenario['P_price'][y] = copy.deepcopy(temp_price)
        scenarios.append(scenario)
    print(" -> 情景生成完毕。")
    return scenarios

# --- 3. 遗传算法核心函数 ---
def create_initial_solution(params):
    solution = {y: {k: {i: None for i in params['I_plots']} for k in [1, 2]} for y in YEARS}
    for y in YEARS:
        for i in params['I_plots']:
            for k in [1, 2]:
                possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
                if possible_crops:
                    solution[y][k][i] = random.choice(possible_crops)
    return repair_solution(solution, params)

def repair_solution(solution, params):
    def get_crops_in_year(sol, y, i):
        crops = set()
        if y == 2023:
            for k in [1, 2]:
                crop = params['P_past'].get(i, {}).get(k)
                if crop: crops.add(crop)
        elif y in sol:
            for k in [1, 2]:
                crop = sol.get(y, {}).get(k, {}).get(i)
                if crop: crops.add(crop)
        return list(crops)
    for i in params['I_plots']:
        for y in YEARS:
            crops_last_year = get_crops_in_year(solution, y - 1, i)
            for k in [1, 2]:
                crop_this_season = solution[y][k][i]
                if crop_this_season and crop_this_season in crops_last_year:
                    possible_replacements = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1 and j not in crops_last_year]
                    solution[y][k][i] = random.choice(possible_replacements) if possible_replacements else None
    for i in params['I_plots']:
        all_years = [2023] + YEARS
        for idx in range(len(all_years) - 2):
            window = all_years[idx:idx+3]
            contains_bean = any(c in params['J_bean'] for y_win in window for c in get_crops_in_year(solution, y_win, i))
            if not contains_bean:
                y_fix = random.choice([y for y in window if y > 2023])
                k_fix = 1
                crops_last_year = get_crops_in_year(solution, y_fix - 1, i)
                possible_beans = [b for b in params['J_bean'] if params['S_suitability'].get((i, b, k_fix), 0) == 1 and b not in crops_last_year]
                if possible_beans:
                    solution[y_fix][k_fix][i] = random.choice(possible_beans)
    return solution

def crossover(p1, p2, params):
    child = copy.deepcopy(p1)
    for i in params['I_plots']:
        if random.random() < 0.5:
            for y in YEARS:
                for k in [1, 2]:
                    child[y][k][i] = p2[y][k][i]
    return child

def mutate(solution, params):
    mut_sol = copy.deepcopy(solution)
    for _ in range(random.randint(1, 5)):
        y = random.choice(YEARS)
        i = random.choice(params['I_plots'])
        k = random.choice([1, 2])
        possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
        if possible_crops:
            mut_sol[y][k][i] = random.choice(possible_crops)
    return mut_sol

def calculate_profits_for_solution(solution, params, scenarios):
    profits = []
    for scenario in scenarios:
        total_revenue, total_cost = 0, 0
        total_production_by_crop = defaultdict(float)
        for y in YEARS:
            for i in params['I_plots']:
                plot_type = params['P_plot_type'][i]
                area = params['P_area'][i]
                for k in [1, 2]:
                    crop = solution[y][k].get(i)
                    if not crop: continue
                    key = (crop, plot_type)
                    cost = scenario['P_cost'][y].get(key, 9e9)
                    yield_val = scenario['P_yield'][y].get(key, 0)
                    if cost > 1e9: continue
                    total_cost += area * cost
                    total_production_by_crop[crop] += area * yield_val
        for crop, production in total_production_by_crop.items():
            total_demand_7_years = sum(scenario['P_demand'][y].get(crop, 0) for y in YEARS)
            all_prices = [p for y_prices in scenario['P_price'].values() for (c, pt), p in y_prices.items() if c == crop and p > 0]
            price = np.mean(all_prices) if all_prices else 0
            if price > 0:
                normal_qty = min(production, total_demand_7_years)
                over_qty = production - normal_qty
                total_revenue += (normal_qty * price) + (over_qty * price * 0.5)
        profits.append(total_revenue - total_cost)
    return profits

def evaluate_fitness_q2(solution, params, scenarios, a_coeff):
    """
    【关键修改】适应度函数：使用 a*E(X) - (1-a)*σ(X)
    """
    profits = calculate_profits_for_solution(solution, params, scenarios)
    valid_profits = [p for p in profits if not np.isnan(p)]
    if not valid_profits: return -1e9

    expected_profit = np.mean(valid_profits)
    risk = np.std(valid_profits)
    
    # a_coeff 是收益的权重, (1-a_coeff) 是风险的权重
    fitness = a_coeff * expected_profit - (1 - a_coeff) * risk
    return fitness

# --- 4. 多种群遗传算法(MPGA)运行器 ---
def run_mpga(params, scenarios, a_coeff):
    print(f"\n--- 开始执行MPGA (风险系数 a = {a_coeff}) ---")
    populations = [[create_initial_solution(params) for _ in range(POP_SIZE_PER_SUBPOP)] for _ in range(NUM_POPULATIONS)]
    best_solution_overall, best_fitness_overall = None, -np.inf
    convergence_history = []
    for gen in tqdm(range(MAX_GEN), desc=f"MPGA进化中 (a={a_coeff})"):
        all_fitnesses = []
        for i in range(NUM_POPULATIONS):
            pop = populations[i]
            fitnesses = [evaluate_fitness_q2(sol, params, scenarios, a_coeff) for sol in pop]
            all_fitnesses.append(fitnesses)
            best_fit_in_pop = np.max(fitnesses)
            if best_fit_in_pop > best_fitness_overall:
                best_fitness_overall = best_fit_in_pop
                best_solution_overall = copy.deepcopy(pop[np.argmax(fitnesses)])
            elite_indices = np.argsort(fitnesses)[-ELITISM_SIZE:]
            new_pop = [pop[idx] for idx in elite_indices]
            while len(new_pop) < POP_SIZE_PER_SUBPOP:
                def tournament_selection(p, f, k):
                    best = random.randrange(len(p))
                    for _ in range(k - 1):
                        idx = random.randrange(len(p))
                        if f[idx] > f[best]: best = idx
                    return p[best]
                p1 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                p2 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                child = crossover(p1, p2, params) if random.random() < CX_PROB else copy.deepcopy(p1)
                if random.random() < MUT_PROB:
                    child = mutate(child, params)
                new_pop.append(repair_solution(child, params))
            populations[i] = new_pop
        if gen > 0 and gen % MIGRATION_INTERVAL == 0:
            for i in range(NUM_POPULATIONS):
                target_pop_idx = (i + 1) % NUM_POPULATIONS
                best_indices_current = np.argsort(all_fitnesses[i])[-MIGRATION_SIZE:]
                migrants = [populations[i][idx] for idx in best_indices_current]
                worst_indices_target = np.argsort(all_fitnesses[target_pop_idx])[:MIGRATION_SIZE]
                for j in range(MIGRATION_SIZE):
                    populations[target_pop_idx][worst_indices_target[j]] = copy.deepcopy(migrants[j])
        convergence_history.append({'Generation': gen, 'Best_Fitness': best_fitness_overall})
    print(f"\n--- MPGA (a={a_coeff}) 优化完成 ---")
    return best_solution_overall, best_fitness_overall, convergence_history

# --- 5. 主程序 ---
if __name__ == '__main__':
    try:
        a = RISK_A_COEFFICIENT
        if not (0.0 <= a <= 1.0):
            raise ValueError("错误：风险系数 a 必须在 0.0 和 1.0 之间。")

        script_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
        data_path = script_dir / '..' / 'Data'
        output_dir = script_dir / 'Result'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"脚本运行目录: {script_dir}")
        print(f"数据读取路径: {data_path}")
        print(f"结果输出路径: {output_dir}")
        
        base_params = load_and_prepare_data(data_path)
        scenarios = generate_scenarios(base_params, NUM_SCENARIOS)

        best_solution, best_fitness, history = run_mpga(base_params, scenarios, a)
        
        print(f"\n求解完成 (风险系数 a = {a})。")
        print(f" -> 最优适应度值: {best_fitness:,.2f}")

        if best_solution:
            file_suffix = f"a{str(a).replace('.', '')}" # e.g., a=0.7 -> a07

            # (1) 保存最终方案
            output_list = []
            for y in sorted(best_solution.keys()):
                for k in sorted(best_solution[y].keys()):
                    for i in sorted(best_solution[y][k].keys()):
                        crop = best_solution[y][k][i]
                        if crop:
                            output_list.append({'年份': y, '季节': k, '地块编号': i, '作物名称': crop, '种植面积（亩）': base_params['P_area'][i]})
            result_df = pd.DataFrame(output_list)
            file_path = output_dir / f'result2_{file_suffix}.xlsx'
            result_df.to_excel(file_path, index=False)
            print(f"最优方案已保存至: {file_path}")

            # (2) 保存收敛过程数据
            history_df = pd.DataFrame(history)
            log_path = output_dir / f'log_{file_suffix}.csv'
            history_df.to_csv(log_path, index=False)
            print(f"收敛过程日志已保存至: {log_path}")

            # (3) 保存最优方案的利润分布
            final_profits = calculate_profits_for_solution(best_solution, base_params, scenarios)
            profits_df = pd.DataFrame(final_profits, columns=['Profit'])
            profits_path = output_dir / f'profits_{file_suffix}.csv'
            profits_df.to_csv(profits_path, index=False)
            print(f"最优方案利润分布已保存至: {profits_path}")
        else:
            print("未能找到有效解。")

    except Exception as e:
        print(f"\n程序主流程发生错误: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# -*- coding: utf-8 -*-
# 文件名: solve_q2_by_risk_coefficient.py
# 功能: 通过调整风险系数a，生成一系列候选最优解。
# 版本: v1.0

import pandas as pd
import numpy as np
import os
import re
import random
import copy
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# --- 1. 【核心配置区】 ---
#  在每次运行时，请修改下面的风险系数 a
#  建议取值: 0.1, 0.3, 0.5, 0.7, 0.9
RISK_A_COEFFICIENT = 0.7  # <--- 在这里修改风险系数a (0.0 到 1.0之间)
# --- ---------------- ---


# (1) 遗传算法参数
POP_SIZE_PER_SUBPOP = 50
NUM_POPULATIONS = 5
MAX_GEN = 200
CX_PROB = 0.8
MUT_PROB = 0.2
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5

# (2) 多种群特定参数
MIGRATION_INTERVAL = 25
MIGRATION_SIZE = 3

# (3) 问题二特定参数
NUM_SCENARIOS = 100
YEARS = list(range(2024, 2031))

# --- 2. 数据加载与情景生成 (函数已折叠，内容与之前版本相同) ---
def load_and_prepare_data(data_path):
    try:
        print("（1）正在读取Excel文件...")
        path_f1 = data_path / '附件1.xlsx'
        path_f2 = data_path / '附件2.xlsx'

        plots_df = pd.read_excel(path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(path_f1, sheet_name='乡村种植的农作物')
        stats_df = pd.read_excel(path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(path_f2, sheet_name='2023年的农作物种植情况')

        for df in [plots_df, crops_info_df, stats_df, past_planting_df]:
            df.columns = df.columns.str.strip()

        params = {}
        params['I_plots'] = sorted(plots_df['地块名称'].tolist())
        params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
        params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))

        params['J_crops'] = sorted(crops_info_df['作物名称'].dropna().unique().tolist())
        params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
        params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and '豆' in ctype]

        params['P_past'] = {i: {1: None, 2: None} for i in params['I_plots']}
        for _, row in past_planting_df.iterrows():
            plot, crop = row['种植地块'], row['作物名称']
            season = row.get('种植季节', 1)
            if plot in params['I_plots']:
                params['P_past'][plot][season] = crop

        def clean_and_convert_price(value):
            if isinstance(value, str) and any(c in value for c in '-–—'):
                parts = re.split(r'[-–—]', value.strip())
                try: return (float(parts[0]) + float(parts[1])) / 2
                except (ValueError, IndexError): return np.nan
            return pd.to_numeric(value, errors='coerce')

        stats_df['销售单价/(元/斤)'] = stats_df['销售单价/(元/斤)'].apply(clean_and_convert_price)
        stats_df.dropna(subset=['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)'], inplace=True)
        
        params['P_yield_base'], params['P_cost_base'], params['P_price_base'] = {}, {}, {}
        for _, row in stats_df.iterrows():
            key = (row['作物名称'], row['地块类型'])
            params['P_cost_base'][key] = row['种植成本/(元/亩)']
            params['P_yield_base'][key] = row['亩产量/斤']
            params['P_price_base'][key] = row['销售单价/(元/斤)']

        params['P_demand_base'] = {j: 0 for j in params['J_crops']}
        merged_df = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称', how='left')
        
        for crop in params['J_crops']:
            crop_plantings = merged_df[merged_df['作物名称'] == crop]
            total_yield = 0
            if not crop_plantings.empty:
                for _, row in crop_plantings.iterrows():
                    area = row.get('种植面积/亩', params['P_area'][row['种植地块']])
                    yield_val = params['P_yield_base'].get((crop, row['地块类型']), 0)
                    total_yield += area * yield_val
            params['P_demand_base'][crop] = total_yield

        params['S_suitability'] = {}
        restricted_veg = ['大白菜', '白萝卜', '红萝卜']
        for i in params['I_plots']:
            plot_t = params['P_plot_type'].get(i, '')
            for j in params['J_crops']:
                crop_t = params['P_crop_type'].get(j, '')
                is_veg = '蔬菜' in str(crop_t)
                for k in [1, 2]:
                    suitable = 0
                    if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in str(crop_t) or j in params['J_bean']) and k == 1: suitable = 1
                    elif plot_t == '水浇地':
                        if '水稻' in j and k == 1: suitable = 1
                        elif is_veg:
                            if j not in restricted_veg and k == 1: suitable = 1
                            elif j in restricted_veg and k == 2: suitable = 1
                    elif plot_t == '普通大棚':
                        if is_veg and j not in restricted_veg and k == 1: suitable = 1
                        elif '食用菌' in str(crop_t) and k == 2: suitable = 1
                    elif plot_t == '智慧大棚' and is_veg and j not in restricted_veg: suitable = 1
                    params['S_suitability'][(i, j, k)] = suitable
        
        print(" -> 基础数据参数准备完成。")
        return params
    except Exception as e:
        print(f"错误: 加载数据失败: {e}"); raise

def generate_scenarios(params, num_scenarios):
    print(f"（2）正在生成 {num_scenarios} 个未来情景...")
    scenarios = []
    crop_types = params['P_crop_type']
    base_demand = params['P_demand_base']
    base_yield = params['P_yield_base']
    base_cost = params['P_cost_base']
    base_price = params['P_price_base']
    for _ in tqdm(range(num_scenarios), desc="生成情景"):
        scenario = {
            'P_demand': {y: {} for y in YEARS}, 'P_yield': {y: {} for y in YEARS},
            'P_cost': {y: {} for y in YEARS}, 'P_price': {y: {} for y in YEARS},
        }
        temp_demand = copy.deepcopy(base_demand)
        temp_cost = copy.deepcopy(base_cost)
        temp_price = copy.deepcopy(base_price)
        for y in YEARS:
            for crop in params['J_crops']:
                if y > 2024:
                    if crop in ['小麦', '玉米']: growth_rate = np.random.uniform(0.05, 0.10)
                    else: growth_rate = np.random.uniform(-0.05, 0.05)
                    temp_demand[crop] *= (1 + growth_rate)
            scenario['P_demand'][y] = copy.deepcopy(temp_demand)
            for key, b_yield in base_yield.items():
                scenario['P_yield'][y][key] = b_yield * (1 + np.random.uniform(-0.10, 0.10))
            if y > 2024:
                for key in temp_cost: temp_cost[key] *= (1 + np.random.normal(0.05, 0.01))
            scenario['P_cost'][y] = copy.deepcopy(temp_cost)
            if y > 2024:
                for key in temp_price:
                    crop, _ = key
                    ctype = crop_types.get(crop, '')
                    if '蔬菜' in str(ctype): temp_price[key] *= (1 + np.random.normal(0.05, 0.02))
                    elif '食用菌' in str(ctype): temp_price[key] *= (1 - (0.05 if crop == '羊肚菌' else np.random.uniform(0.01, 0.05)))
            scenario['P_price'][y] = copy.deepcopy(temp_price)
        scenarios.append(scenario)
    print(" -> 情景生成完毕。")
    return scenarios

# --- 3. 遗传算法核心函数 ---
def create_initial_solution(params):
    solution = {y: {k: {i: None for i in params['I_plots']} for k in [1, 2]} for y in YEARS}
    for y in YEARS:
        for i in params['I_plots']:
            for k in [1, 2]:
                possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
                if possible_crops:
                    solution[y][k][i] = random.choice(possible_crops)
    return repair_solution(solution, params)

def repair_solution(solution, params):
    def get_crops_in_year(sol, y, i):
        crops = set()
        if y == 2023:
            for k in [1, 2]:
                crop = params['P_past'].get(i, {}).get(k)
                if crop: crops.add(crop)
        elif y in sol:
            for k in [1, 2]:
                crop = sol.get(y, {}).get(k, {}).get(i)
                if crop: crops.add(crop)
        return list(crops)
    for i in params['I_plots']:
        for y in YEARS:
            crops_last_year = get_crops_in_year(solution, y - 1, i)
            for k in [1, 2]:
                crop_this_season = solution[y][k][i]
                if crop_this_season and crop_this_season in crops_last_year:
                    possible_replacements = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1 and j not in crops_last_year]
                    solution[y][k][i] = random.choice(possible_replacements) if possible_replacements else None
    for i in params['I_plots']:
        all_years = [2023] + YEARS
        for idx in range(len(all_years) - 2):
            window = all_years[idx:idx+3]
            contains_bean = any(c in params['J_bean'] for y_win in window for c in get_crops_in_year(solution, y_win, i))
            if not contains_bean:
                y_fix = random.choice([y for y in window if y > 2023])
                k_fix = 1
                crops_last_year = get_crops_in_year(solution, y_fix - 1, i)
                possible_beans = [b for b in params['J_bean'] if params['S_suitability'].get((i, b, k_fix), 0) == 1 and b not in crops_last_year]
                if possible_beans:
                    solution[y_fix][k_fix][i] = random.choice(possible_beans)
    return solution

def crossover(p1, p2, params):
    child = copy.deepcopy(p1)
    for i in params['I_plots']:
        if random.random() < 0.5:
            for y in YEARS:
                for k in [1, 2]:
                    child[y][k][i] = p2[y][k][i]
    return child

def mutate(solution, params):
    mut_sol = copy.deepcopy(solution)
    for _ in range(random.randint(1, 5)):
        y = random.choice(YEARS)
        i = random.choice(params['I_plots'])
        k = random.choice([1, 2])
        possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
        if possible_crops:
            mut_sol[y][k][i] = random.choice(possible_crops)
    return mut_sol

def calculate_profits_for_solution(solution, params, scenarios):
    profits = []
    for scenario in scenarios:
        total_revenue, total_cost = 0, 0
        total_production_by_crop = defaultdict(float)
        for y in YEARS:
            for i in params['I_plots']:
                plot_type = params['P_plot_type'][i]
                area = params['P_area'][i]
                for k in [1, 2]:
                    crop = solution[y][k].get(i)
                    if not crop: continue
                    key = (crop, plot_type)
                    cost = scenario['P_cost'][y].get(key, 9e9)
                    yield_val = scenario['P_yield'][y].get(key, 0)
                    if cost > 1e9: continue
                    total_cost += area * cost
                    total_production_by_crop[crop] += area * yield_val
        for crop, production in total_production_by_crop.items():
            total_demand_7_years = sum(scenario['P_demand'][y].get(crop, 0) for y in YEARS)
            all_prices = [p for y_prices in scenario['P_price'].values() for (c, pt), p in y_prices.items() if c == crop and p > 0]
            price = np.mean(all_prices) if all_prices else 0
            if price > 0:
                normal_qty = min(production, total_demand_7_years)
                over_qty = production - normal_qty
                total_revenue += (normal_qty * price) + (over_qty * price * 0.5)
        profits.append(total_revenue - total_cost)
    return profits

def evaluate_fitness_q2(solution, params, scenarios, a_coeff):
    """
    【关键修改】适应度函数：使用 a*E(X) - (1-a)*σ(X)
    """
    profits = calculate_profits_for_solution(solution, params, scenarios)
    valid_profits = [p for p in profits if not np.isnan(p)]
    if not valid_profits: return -1e9

    expected_profit = np.mean(valid_profits)
    risk = np.std(valid_profits)
    
    # a_coeff 是收益的权重, (1-a_coeff) 是风险的权重
    fitness = a_coeff * expected_profit - (1 - a_coeff) * risk
    return fitness

# --- 4. 多种群遗传算法(MPGA)运行器 ---
def run_mpga(params, scenarios, a_coeff):
    print(f"\n--- 开始执行MPGA (风险系数 a = {a_coeff}) ---")
    populations = [[create_initial_solution(params) for _ in range(POP_SIZE_PER_SUBPOP)] for _ in range(NUM_POPULATIONS)]
    best_solution_overall, best_fitness_overall = None, -np.inf
    convergence_history = []
    for gen in tqdm(range(MAX_GEN), desc=f"MPGA进化中 (a={a_coeff})"):
        all_fitnesses = []
        for i in range(NUM_POPULATIONS):
            pop = populations[i]
            fitnesses = [evaluate_fitness_q2(sol, params, scenarios, a_coeff) for sol in pop]
            all_fitnesses.append(fitnesses)
            best_fit_in_pop = np.max(fitnesses)
            if best_fit_in_pop > best_fitness_overall:
                best_fitness_overall = best_fit_in_pop
                best_solution_overall = copy.deepcopy(pop[np.argmax(fitnesses)])
            elite_indices = np.argsort(fitnesses)[-ELITISM_SIZE:]
            new_pop = [pop[idx] for idx in elite_indices]
            while len(new_pop) < POP_SIZE_PER_SUBPOP:
                def tournament_selection(p, f, k):
                    best = random.randrange(len(p))
                    for _ in range(k - 1):
                        idx = random.randrange(len(p))
                        if f[idx] > f[best]: best = idx
                    return p[best]
                p1 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                p2 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                child = crossover(p1, p2, params) if random.random() < CX_PROB else copy.deepcopy(p1)
                if random.random() < MUT_PROB:
                    child = mutate(child, params)
                new_pop.append(repair_solution(child, params))
            populations[i] = new_pop
        if gen > 0 and gen % MIGRATION_INTERVAL == 0:
            for i in range(NUM_POPULATIONS):
                target_pop_idx = (i + 1) % NUM_POPULATIONS
                best_indices_current = np.argsort(all_fitnesses[i])[-MIGRATION_SIZE:]
                migrants = [populations[i][idx] for idx in best_indices_current]
                worst_indices_target = np.argsort(all_fitnesses[target_pop_idx])[:MIGRATION_SIZE]
                for j in range(MIGRATION_SIZE):
                    populations[target_pop_idx][worst_indices_target[j]] = copy.deepcopy(migrants[j])
        convergence_history.append({'Generation': gen, 'Best_Fitness': best_fitness_overall})
    print(f"\n--- MPGA (a={a_coeff}) 优化完成 ---")
    return best_solution_overall, best_fitness_overall, convergence_history

# --- 5. 主程序 ---
if __name__ == '__main__':
    try:
        a = RISK_A_COEFFICIENT
        if not (0.0 <= a <= 1.0):
            raise ValueError("错误：风险系数 a 必须在 0.0 和 1.0 之间。")

        script_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
        data_path = script_dir / '..' / 'Data'
        output_dir = script_dir / 'Result'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"脚本运行目录: {script_dir}")
        print(f"数据读取路径: {data_path}")
        print(f"结果输出路径: {output_dir}")
        
        base_params = load_and_prepare_data(data_path)
        scenarios = generate_scenarios(base_params, NUM_SCENARIOS)

        best_solution, best_fitness, history = run_mpga(base_params, scenarios, a)
        
        print(f"\n求解完成 (风险系数 a = {a})。")
        print(f" -> 最优适应度值: {best_fitness:,.2f}")

        if best_solution:
            file_suffix = f"a{str(a).replace('.', '')}" # e.g., a=0.7 -> a07

            # (1) 保存最终方案
            output_list = []
            for y in sorted(best_solution.keys()):
                for k in sorted(best_solution[y].keys()):
                    for i in sorted(best_solution[y][k].keys()):
                        crop = best_solution[y][k][i]
                        if crop:
                            output_list.append({'年份': y, '季节': k, '地块编号': i, '作物名称': crop, '种植面积（亩）': base_params['P_area'][i]})
            result_df = pd.DataFrame(output_list)
            file_path = output_dir / f'result2_{file_suffix}.xlsx'
            result_df.to_excel(file_path, index=False)
            print(f"最优方案已保存至: {file_path}")

            # (2) 保存收敛过程数据
            history_df = pd.DataFrame(history)
            log_path = output_dir / f'log_{file_suffix}.csv'
            history_df.to_csv(log_path, index=False)
            print(f"收敛过程日志已保存至: {log_path}")

            # (3) 保存最优方案的利润分布
            final_profits = calculate_profits_for_solution(best_solution, base_params, scenarios)
            profits_df = pd.DataFrame(final_profits, columns=['Profit'])
            profits_path = output_dir / f'profits_{file_suffix}.csv'
            profits_df.to_csv(profits_path, index=False)
            print(f"最优方案利润分布已保存至: {profits_path}")
        else:
            print("未能找到有效解。")

    except Exception as e:
        print(f"\n程序主流程发生错误: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# -*- coding: utf-8 -*-
# 文件名: solve_q2_by_risk_coefficient.py
# 功能: 通过调整风险系数a，生成一系列候选最优解。
# 版本: v1.0

import pandas as pd
import numpy as np
import os
import re
import random
import copy
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# --- 1. 【核心配置区】 ---
#  在每次运行时，请修改下面的风险系数 a
#  建议取值: 0.1, 0.3, 0.5, 0.7, 0.9
RISK_A_COEFFICIENT = 0.9  # <--- 在这里修改风险系数a (0.0 到 1.0之间)
# --- ---------------- ---


# (1) 遗传算法参数
POP_SIZE_PER_SUBPOP = 50
NUM_POPULATIONS = 5
MAX_GEN = 200
CX_PROB = 0.8
MUT_PROB = 0.2
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5

# (2) 多种群特定参数
MIGRATION_INTERVAL = 25
MIGRATION_SIZE = 3

# (3) 问题二特定参数
NUM_SCENARIOS = 100
YEARS = list(range(2024, 2031))

# --- 2. 数据加载与情景生成 (函数已折叠，内容与之前版本相同) ---
def load_and_prepare_data(data_path):
    try:
        print("（1）正在读取Excel文件...")
        path_f1 = data_path / '附件1.xlsx'
        path_f2 = data_path / '附件2.xlsx'

        plots_df = pd.read_excel(path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(path_f1, sheet_name='乡村种植的农作物')
        stats_df = pd.read_excel(path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(path_f2, sheet_name='2023年的农作物种植情况')

        for df in [plots_df, crops_info_df, stats_df, past_planting_df]:
            df.columns = df.columns.str.strip()

        params = {}
        params['I_plots'] = sorted(plots_df['地块名称'].tolist())
        params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
        params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))

        params['J_crops'] = sorted(crops_info_df['作物名称'].dropna().unique().tolist())
        params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
        params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and '豆' in ctype]

        params['P_past'] = {i: {1: None, 2: None} for i in params['I_plots']}
        for _, row in past_planting_df.iterrows():
            plot, crop = row['种植地块'], row['作物名称']
            season = row.get('种植季节', 1)
            if plot in params['I_plots']:
                params['P_past'][plot][season] = crop

        def clean_and_convert_price(value):
            if isinstance(value, str) and any(c in value for c in '-–—'):
                parts = re.split(r'[-–—]', value.strip())
                try: return (float(parts[0]) + float(parts[1])) / 2
                except (ValueError, IndexError): return np.nan
            return pd.to_numeric(value, errors='coerce')

        stats_df['销售单价/(元/斤)'] = stats_df['销售单价/(元/斤)'].apply(clean_and_convert_price)
        stats_df.dropna(subset=['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)'], inplace=True)
        
        params['P_yield_base'], params['P_cost_base'], params['P_price_base'] = {}, {}, {}
        for _, row in stats_df.iterrows():
            key = (row['作物名称'], row['地块类型'])
            params['P_cost_base'][key] = row['种植成本/(元/亩)']
            params['P_yield_base'][key] = row['亩产量/斤']
            params['P_price_base'][key] = row['销售单价/(元/斤)']

        params['P_demand_base'] = {j: 0 for j in params['J_crops']}
        merged_df = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称', how='left')
        
        for crop in params['J_crops']:
            crop_plantings = merged_df[merged_df['作物名称'] == crop]
            total_yield = 0
            if not crop_plantings.empty:
                for _, row in crop_plantings.iterrows():
                    area = row.get('种植面积/亩', params['P_area'][row['种植地块']])
                    yield_val = params['P_yield_base'].get((crop, row['地块类型']), 0)
                    total_yield += area * yield_val
            params['P_demand_base'][crop] = total_yield

        params['S_suitability'] = {}
        restricted_veg = ['大白菜', '白萝卜', '红萝卜']
        for i in params['I_plots']:
            plot_t = params['P_plot_type'].get(i, '')
            for j in params['J_crops']:
                crop_t = params['P_crop_type'].get(j, '')
                is_veg = '蔬菜' in str(crop_t)
                for k in [1, 2]:
                    suitable = 0
                    if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in str(crop_t) or j in params['J_bean']) and k == 1: suitable = 1
                    elif plot_t == '水浇地':
                        if '水稻' in j and k == 1: suitable = 1
                        elif is_veg:
                            if j not in restricted_veg and k == 1: suitable = 1
                            elif j in restricted_veg and k == 2: suitable = 1
                    elif plot_t == '普通大棚':
                        if is_veg and j not in restricted_veg and k == 1: suitable = 1
                        elif '食用菌' in str(crop_t) and k == 2: suitable = 1
                    elif plot_t == '智慧大棚' and is_veg and j not in restricted_veg: suitable = 1
                    params['S_suitability'][(i, j, k)] = suitable
        
        print(" -> 基础数据参数准备完成。")
        return params
    except Exception as e:
        print(f"错误: 加载数据失败: {e}"); raise

def generate_scenarios(params, num_scenarios):
    print(f"（2）正在生成 {num_scenarios} 个未来情景...")
    scenarios = []
    crop_types = params['P_crop_type']
    base_demand = params['P_demand_base']
    base_yield = params['P_yield_base']
    base_cost = params['P_cost_base']
    base_price = params['P_price_base']
    for _ in tqdm(range(num_scenarios), desc="生成情景"):
        scenario = {
            'P_demand': {y: {} for y in YEARS}, 'P_yield': {y: {} for y in YEARS},
            'P_cost': {y: {} for y in YEARS}, 'P_price': {y: {} for y in YEARS},
        }
        temp_demand = copy.deepcopy(base_demand)
        temp_cost = copy.deepcopy(base_cost)
        temp_price = copy.deepcopy(base_price)
        for y in YEARS:
            for crop in params['J_crops']:
                if y > 2024:
                    if crop in ['小麦', '玉米']: growth_rate = np.random.uniform(0.05, 0.10)
                    else: growth_rate = np.random.uniform(-0.05, 0.05)
                    temp_demand[crop] *= (1 + growth_rate)
            scenario['P_demand'][y] = copy.deepcopy(temp_demand)
            for key, b_yield in base_yield.items():
                scenario['P_yield'][y][key] = b_yield * (1 + np.random.uniform(-0.10, 0.10))
            if y > 2024:
                for key in temp_cost: temp_cost[key] *= (1 + np.random.normal(0.05, 0.01))
            scenario['P_cost'][y] = copy.deepcopy(temp_cost)
            if y > 2024:
                for key in temp_price:
                    crop, _ = key
                    ctype = crop_types.get(crop, '')
                    if '蔬菜' in str(ctype): temp_price[key] *= (1 + np.random.normal(0.05, 0.02))
                    elif '食用菌' in str(ctype): temp_price[key] *= (1 - (0.05 if crop == '羊肚菌' else np.random.uniform(0.01, 0.05)))
            scenario['P_price'][y] = copy.deepcopy(temp_price)
        scenarios.append(scenario)
    print(" -> 情景生成完毕。")
    return scenarios

# --- 3. 遗传算法核心函数 ---
def create_initial_solution(params):
    solution = {y: {k: {i: None for i in params['I_plots']} for k in [1, 2]} for y in YEARS}
    for y in YEARS:
        for i in params['I_plots']:
            for k in [1, 2]:
                possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
                if possible_crops:
                    solution[y][k][i] = random.choice(possible_crops)
    return repair_solution(solution, params)

def repair_solution(solution, params):
    def get_crops_in_year(sol, y, i):
        crops = set()
        if y == 2023:
            for k in [1, 2]:
                crop = params['P_past'].get(i, {}).get(k)
                if crop: crops.add(crop)
        elif y in sol:
            for k in [1, 2]:
                crop = sol.get(y, {}).get(k, {}).get(i)
                if crop: crops.add(crop)
        return list(crops)
    for i in params['I_plots']:
        for y in YEARS:
            crops_last_year = get_crops_in_year(solution, y - 1, i)
            for k in [1, 2]:
                crop_this_season = solution[y][k][i]
                if crop_this_season and crop_this_season in crops_last_year:
                    possible_replacements = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1 and j not in crops_last_year]
                    solution[y][k][i] = random.choice(possible_replacements) if possible_replacements else None
    for i in params['I_plots']:
        all_years = [2023] + YEARS
        for idx in range(len(all_years) - 2):
            window = all_years[idx:idx+3]
            contains_bean = any(c in params['J_bean'] for y_win in window for c in get_crops_in_year(solution, y_win, i))
            if not contains_bean:
                y_fix = random.choice([y for y in window if y > 2023])
                k_fix = 1
                crops_last_year = get_crops_in_year(solution, y_fix - 1, i)
                possible_beans = [b for b in params['J_bean'] if params['S_suitability'].get((i, b, k_fix), 0) == 1 and b not in crops_last_year]
                if possible_beans:
                    solution[y_fix][k_fix][i] = random.choice(possible_beans)
    return solution

def crossover(p1, p2, params):
    child = copy.deepcopy(p1)
    for i in params['I_plots']:
        if random.random() < 0.5:
            for y in YEARS:
                for k in [1, 2]:
                    child[y][k][i] = p2[y][k][i]
    return child

def mutate(solution, params):
    mut_sol = copy.deepcopy(solution)
    for _ in range(random.randint(1, 5)):
        y = random.choice(YEARS)
        i = random.choice(params['I_plots'])
        k = random.choice([1, 2])
        possible_crops = [j for j in params['J_crops'] if params['S_suitability'].get((i, j, k), 0) == 1]
        if possible_crops:
            mut_sol[y][k][i] = random.choice(possible_crops)
    return mut_sol

def calculate_profits_for_solution(solution, params, scenarios):
    profits = []
    for scenario in scenarios:
        total_revenue, total_cost = 0, 0
        total_production_by_crop = defaultdict(float)
        for y in YEARS:
            for i in params['I_plots']:
                plot_type = params['P_plot_type'][i]
                area = params['P_area'][i]
                for k in [1, 2]:
                    crop = solution[y][k].get(i)
                    if not crop: continue
                    key = (crop, plot_type)
                    cost = scenario['P_cost'][y].get(key, 9e9)
                    yield_val = scenario['P_yield'][y].get(key, 0)
                    if cost > 1e9: continue
                    total_cost += area * cost
                    total_production_by_crop[crop] += area * yield_val
        for crop, production in total_production_by_crop.items():
            total_demand_7_years = sum(scenario['P_demand'][y].get(crop, 0) for y in YEARS)
            all_prices = [p for y_prices in scenario['P_price'].values() for (c, pt), p in y_prices.items() if c == crop and p > 0]
            price = np.mean(all_prices) if all_prices else 0
            if price > 0:
                normal_qty = min(production, total_demand_7_years)
                over_qty = production - normal_qty
                total_revenue += (normal_qty * price) + (over_qty * price * 0.5)
        profits.append(total_revenue - total_cost)
    return profits

def evaluate_fitness_q2(solution, params, scenarios, a_coeff):
    """
    【关键修改】适应度函数：使用 a*E(X) - (1-a)*σ(X)
    """
    profits = calculate_profits_for_solution(solution, params, scenarios)
    valid_profits = [p for p in profits if not np.isnan(p)]
    if not valid_profits: return -1e9

    expected_profit = np.mean(valid_profits)
    risk = np.std(valid_profits)
    
    # a_coeff 是收益的权重, (1-a_coeff) 是风险的权重
    fitness = a_coeff * expected_profit - (1 - a_coeff) * risk
    return fitness

# --- 4. 多种群遗传算法(MPGA)运行器 ---
def run_mpga(params, scenarios, a_coeff):
    print(f"\n--- 开始执行MPGA (风险系数 a = {a_coeff}) ---")
    populations = [[create_initial_solution(params) for _ in range(POP_SIZE_PER_SUBPOP)] for _ in range(NUM_POPULATIONS)]
    best_solution_overall, best_fitness_overall = None, -np.inf
    convergence_history = []
    for gen in tqdm(range(MAX_GEN), desc=f"MPGA进化中 (a={a_coeff})"):
        all_fitnesses = []
        for i in range(NUM_POPULATIONS):
            pop = populations[i]
            fitnesses = [evaluate_fitness_q2(sol, params, scenarios, a_coeff) for sol in pop]
            all_fitnesses.append(fitnesses)
            best_fit_in_pop = np.max(fitnesses)
            if best_fit_in_pop > best_fitness_overall:
                best_fitness_overall = best_fit_in_pop
                best_solution_overall = copy.deepcopy(pop[np.argmax(fitnesses)])
            elite_indices = np.argsort(fitnesses)[-ELITISM_SIZE:]
            new_pop = [pop[idx] for idx in elite_indices]
            while len(new_pop) < POP_SIZE_PER_SUBPOP:
                def tournament_selection(p, f, k):
                    best = random.randrange(len(p))
                    for _ in range(k - 1):
                        idx = random.randrange(len(p))
                        if f[idx] > f[best]: best = idx
                    return p[best]
                p1 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                p2 = tournament_selection(pop, fitnesses, TOURNAMENT_SIZE)
                child = crossover(p1, p2, params) if random.random() < CX_PROB else copy.deepcopy(p1)
                if random.random() < MUT_PROB:
                    child = mutate(child, params)
                new_pop.append(repair_solution(child, params))
            populations[i] = new_pop
        if gen > 0 and gen % MIGRATION_INTERVAL == 0:
            for i in range(NUM_POPULATIONS):
                target_pop_idx = (i + 1) % NUM_POPULATIONS
                best_indices_current = np.argsort(all_fitnesses[i])[-MIGRATION_SIZE:]
                migrants = [populations[i][idx] for idx in best_indices_current]
                worst_indices_target = np.argsort(all_fitnesses[target_pop_idx])[:MIGRATION_SIZE]
                for j in range(MIGRATION_SIZE):
                    populations[target_pop_idx][worst_indices_target[j]] = copy.deepcopy(migrants[j])
        convergence_history.append({'Generation': gen, 'Best_Fitness': best_fitness_overall})
    print(f"\n--- MPGA (a={a_coeff}) 优化完成 ---")
    return best_solution_overall, best_fitness_overall, convergence_history

# --- 5. 主程序 ---
if __name__ == '__main__':
    try:
        a = RISK_A_COEFFICIENT
        if not (0.0 <= a <= 1.0):
            raise ValueError("错误：风险系数 a 必须在 0.0 和 1.0 之间。")

        script_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
        data_path = script_dir / '..' / 'Data'
        output_dir = script_dir / 'Result'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"脚本运行目录: {script_dir}")
        print(f"数据读取路径: {data_path}")
        print(f"结果输出路径: {output_dir}")
        
        base_params = load_and_prepare_data(data_path)
        scenarios = generate_scenarios(base_params, NUM_SCENARIOS)

        best_solution, best_fitness, history = run_mpga(base_params, scenarios, a)
        
        print(f"\n求解完成 (风险系数 a = {a})。")
        print(f" -> 最优适应度值: {best_fitness:,.2f}")

        if best_solution:
            file_suffix = f"a{str(a).replace('.', '')}" # e.g., a=0.7 -> a07

            # (1) 保存最终方案
            output_list = []
            for y in sorted(best_solution.keys()):
                for k in sorted(best_solution[y].keys()):
                    for i in sorted(best_solution[y][k].keys()):
                        crop = best_solution[y][k][i]
                        if crop:
                            output_list.append({'年份': y, '季节': k, '地块编号': i, '作物名称': crop, '种植面积（亩）': base_params['P_area'][i]})
            result_df = pd.DataFrame(output_list)
            file_path = output_dir / f'result2_{file_suffix}.xlsx'
            result_df.to_excel(file_path, index=False)
            print(f"最优方案已保存至: {file_path}")

            # (2) 保存收敛过程数据
            history_df = pd.DataFrame(history)
            log_path = output_dir / f'log_{file_suffix}.csv'
            history_df.to_csv(log_path, index=False)
            print(f"收敛过程日志已保存至: {log_path}")

            # (3) 保存最优方案的利润分布
            final_profits = calculate_profits_for_solution(best_solution, base_params, scenarios)
            profits_df = pd.DataFrame(final_profits, columns=['Profit'])
            profits_path = output_dir / f'profits_{file_suffix}.csv'
            profits_df.to_csv(profits_path, index=False)
            print(f"最优方案利润分布已保存至: {profits_path}")
        else:
            print("未能找到有效解。")

    except Exception as e:
        print(f"\n程序主流程发生错误: {e}")
        import traceback
        traceback.print_exc()